In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")

C:\Users\razas\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
file_path = r"C:\Users\razas\Downloads\TechnoHacks Internship\Technohack Machine Learning\Movie review classification\labeledTrainData.tsv\labeledTrainData.tsv"
df = pd.read_csv(file_path, sep="\t", quoting=3)


In [3]:
df

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
...,...,...,...
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."


In [4]:
df.to_csv("movie_reviews.csv", index=False)


In [5]:
file_path = r"C:\Users\razas\Downloads\TechnoHacks Internship\Technohack Machine Learning\Movie review classification\unlabeledTrainData.tsv\unlabeledTrainData.tsv"
df2 = pd.read_csv(file_path, sep="\t", quoting=3)


In [6]:
df2

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."
...,...,...
49995,"""18984_0""","""The original Man Eater by Joe D'Amato is some..."
49996,"""16433_0""","""When Home Box Office was in it's early days m..."
49997,"""16006_0""","""Griffin Dunne was born into a cultural family..."
49998,"""40155_0""","""Not a bad story, but the low budget rears its..."


In [7]:
df2.to_csv("movie_reviews_unclass.csv", index=False)

#    PREPROCESSING

In [8]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('punkt')            # Tokenization
nltk.download('stopwords')        # Stopwords
nltk.download('wordnet')          # Lemmatization
nltk.download('omw-1.4')          # WordNet lemmatizer dependency


# Define preprocessing function
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()  # Convert to lowercase
        text = re.sub(r'<.*?>', ' ', text)  # Remove HTML tags
        text = re.sub(r'\d+', '', text)  # Remove numbers
        text = re.sub(r'[^\w\s]', ' ', text)  # Remove punctuation
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
        
        tokens = nltk.word_tokenize(text)  # Tokenize
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]  # Remove stop words
        
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatize
        
        return " ".join(tokens)
    else:
        return ""

# Apply preprocessing to the 'review' column
df['cleaned_review'] = df['review'].apply(preprocess_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\razas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\razas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\razas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\razas\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [9]:

from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['cleaned_review'])

# Target Variable
Y = df['sentiment']  # assuming 0 = negative, 1 = positive

In [10]:
df.columns

Index(['id', 'sentiment', 'review', 'cleaned_review'], dtype='object')

In [11]:
Y=df['sentiment']

In [12]:
from sklearn.model_selection import train_test_split
#split the data into test and train 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3,random_state=10)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Train the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, Y_train)

# Predict on the test set
Y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')  # use 'macro' or 'binary' as per need
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

# Print evaluation metrics
print(f"Random Forest Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(Y_test, Y_pred))

# Compute and print the confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:")
print(cm)

Random Forest Accuracy: 0.84
Precision: 0.84
Recall: 0.84
F1-Score: 0.84

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.85      0.84      3701
           1       0.85      0.83      0.84      3799

    accuracy                           0.84      7500
   macro avg       0.84      0.84      0.84      7500
weighted avg       0.84      0.84      0.84      7500

Confusion Matrix:
[[3151  550]
 [ 661 3138]]


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Train the Logistic Regression model
log_model = LogisticRegression(max_iter=1000, random_state=42)
log_model.fit(X_train, Y_train)

# Predict on the test set
Y_pred = log_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

# Print evaluation metrics
print("=== Logistic Regression ===")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(Y_test, Y_pred))

# Confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:")
print(cm)


=== Logistic Regression ===
Accuracy: 0.88
Precision: 0.88
Recall: 0.88
F1-Score: 0.88

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      3701
           1       0.88      0.90      0.89      3799

    accuracy                           0.88      7500
   macro avg       0.88      0.88      0.88      7500
weighted avg       0.88      0.88      0.88      7500

Confusion Matrix:
[[3228  473]
 [ 395 3404]]


# logistic regression is the best model

In [15]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Train the Linear Support Vector Classifier
svc_model = LinearSVC(random_state=42)
svc_model.fit(X_train, Y_train)

# Predict on the test set
Y_pred = svc_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

# Print evaluation metrics
print("=== Linear SVC ===")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(Y_test, Y_pred))

# Confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:")
print(cm)


=== Linear SVC ===
Accuracy: 0.88
Precision: 0.88
Recall: 0.88
F1-Score: 0.88

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.87      0.87      3701
           1       0.87      0.88      0.88      3799

    accuracy                           0.88      7500
   macro avg       0.88      0.88      0.88      7500
weighted avg       0.88      0.88      0.88      7500

Confusion Matrix:
[[3219  482]
 [ 447 3352]]


In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, Y_train)

# Predict on the test set
Y_pred = knn_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

# Print evaluation metrics
print("=== K-Nearest Neighbors (KNN) ===")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(Y_test, Y_pred))

# Confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:")
print(cm)


=== K-Nearest Neighbors (KNN) ===
Accuracy: 0.76
Precision: 0.77
Recall: 0.76
F1-Score: 0.76

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.70      0.75      3701
           1       0.74      0.83      0.78      3799

    accuracy                           0.76      7500
   macro avg       0.77      0.76      0.76      7500
weighted avg       0.77      0.76      0.76      7500

Confusion Matrix:
[[2591 1110]
 [ 663 3136]]


In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Train the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, Y_train)

# Predict on the test set
Y_pred = dt_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

# Print evaluation metrics
print("=== Decision Tree Classifier ===")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(Y_test, Y_pred))

# Confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:")
print(cm)


=== Decision Tree Classifier ===
Accuracy: 0.71
Precision: 0.71
Recall: 0.71
F1-Score: 0.71

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.71      0.71      3701
           1       0.72      0.71      0.71      3799

    accuracy                           0.71      7500
   macro avg       0.71      0.71      0.71      7500
weighted avg       0.71      0.71      0.71      7500

Confusion Matrix:
[[2621 1080]
 [1087 2712]]


In [18]:
import joblib

# Save the trained model to a file
joblib.dump(log_model, 'logistic_regression_model.pkl')



['logistic_regression_model.pkl']

In [19]:
file_path = r"C:\Users\razas\Downloads\TechnoHacks Internship\Technohack Machine Learning\Movie review classification\unlabeledTrainData.tsv\unlabeledTrainData.tsv"
df2 = pd.read_csv(file_path, sep="\t", quoting=3)
df2

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."
...,...,...
49995,"""18984_0""","""The original Man Eater by Joe D'Amato is some..."
49996,"""16433_0""","""When Home Box Office was in it's early days m..."
49997,"""16006_0""","""Griffin Dunne was born into a cultural family..."
49998,"""40155_0""","""Not a bad story, but the low budget rears its..."


In [20]:
# Preprocess df2 reviews
df2['cleaned_review'] = df2['review'].apply(preprocess_text)

# Vectorize using the trained vectorizer (same as used on X_train)
df2_vectorized = vectorizer.transform(df2['cleaned_review'])

# Predict using the trained model
df2['predicted_sentiment'] = log_model.predict(df2_vectorized)

# Convert numerical prediction to label
df2['predicted_label'] = df2['predicted_sentiment'].apply(lambda x: 'Positive' if x == 1 else 'Negative')

# View result
print(df2[['review', 'id','predicted_label']].head())

# Optional: Save predictions to CSV
df2.to_csv('df2_sentiment_predictions.csv', index=False)


                                              review         id  \
0  "Watching Time Chasers, it obvious that it was...   "9999_0"   
1  "I saw this film about 20 years ago and rememb...  "45057_0"   
2  "Minor Spoilers<br /><br />In New York, Joan B...  "15561_0"   
3  "I went to see this film with a great deal of ...   "7161_0"   
4  "Yes, I agree with everyone on this site this ...  "43971_0"   

  predicted_label  
0        Negative  
1        Negative  
2        Negative  
3        Negative  
4        Negative  


In [21]:


file_path = r"C:\Users\razas\Downloads\TechnoHacks Internship\Technohack Machine Learning\Movie review classification\testData.tsv\testData.tsv"

# Read the TSV file (without headers)
df3 = pd.read_csv(file_path, sep='\t', quoting=3, header=None, names=["id", "review"])

# Preview the DataFrame
df3.head()


,id,review
0,id,review
1,"""12311_10""","""Naturally in a film who's main themes are of ..."
2,"""8348_2""","""This movie is a disaster within a disaster fi..."
3,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
4,"""7186_2""","""Afraid of the Dark left me with the impressio..."


In [22]:
df3.to_csv("testdata.csv",index=False)


In [23]:
# Preprocess df2 reviews
df3['cleaned_review'] = df3['review'].apply(preprocess_text)

# Vectorize using the trained vectorizer (same as used on X_train)
df3_vectorized = vectorizer.transform(df3['cleaned_review'])

# Predict using the trained model
df3['predicted_sentiment'] = log_model.predict(df3_vectorized)

# Convert numerical prediction to label
df3['predicted_label'] = df3['predicted_sentiment'].apply(lambda x: 'Positive' if x == 1 else 'Negative')

# View result
print(df3[['review', 'id','predicted_label']].head())

# Optional: Save predictions to CSV
df3.to_csv('df3_sentiment_predictions.csv', index=False)


                                              review          id  \
0                                             review          id   
1  "Naturally in a film who's main themes are of ...  "12311_10"   
2  "This movie is a disaster within a disaster fi...    "8348_2"   
3  "All in all, this is a movie for kids. We saw ...    "5828_4"   
4  "Afraid of the Dark left me with the impressio...    "7186_2"   

  predicted_label  
0        Negative  
1        Positive  
2        Negative  
3        Positive  
4        Positive  


# project ends here